In [18]:
import datetime
import numpy as np
import sys
import os
import xarray as xr

sys.path.append(os.environ.get('srcdir'))
# always reload modules marked with "%aimport"
%load_ext autoreload
%autoreload 1

%aimport features.compare_tide
from features.compare_tide import compare_tide

%aimport features.compare_atg
from features.compare_atg import compare_atg
%aimport features.grid_ttide
from features.grid_ttide import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
#load data at certain time range
file_path = os.path.join(os.environ.get('rawdir'),'waom10_v2.0_small','ocean_his_hourly_0001.nc')
zeta = xr.open_dataset(file_path).zeta.sel(ocean_time=slice('2007-1-15','2007-2-15'))

#load grid
file_path = os.path.join(os.environ.get('rawdir'),'gdata','waom10_v2.0_frc','waom10_small_grd.nc')
grid = xr.open_dataset(file_path)

In [108]:
#define functions that calculate rmsd to atg
from scipy.spatial import KDTree
import ttide as tt
import scipy.io as sio
from features.log_progress import log_progress
from features.haversine import haversine

def compare_atg(roms_zeta_da,grid,
                atg_mat_path=os.path.join(os.environ.get('projdir'),'data','analysis','external','atg','ATG_ocean_height_2010_0908.mat'),
                stime=datetime.datetime(2007,1,1),constit_list = ['M2','O1'],
                station_list=np.arange(1,109),print_flag=True):

    print('stime = ',stime,' constits = ',constit_list,'stations = ',station_list)
    
    mat_content = sio.loadmat(atg_mat_path)
    atg_data = mat_content['atg']

    station_dict = {}

    for station in log_progress(station_list,name='stations'):

        station_dict[station] = {}

        atg_dict = read_atg(atg_data,station,constit_list)
        lat = atg_dict['lat']
        lon = atg_dict['lon']
        eta_rho,xi_rho,dist,tt_dict = station_ttide(roms_zeta_da,grid,lat,lon,stime,constit_list)

        station_dict[station]['atg'] = atg_dict
        station_dict[station]['tt'] = tt_dict
        station_dict[station]['dist'] = dist
        station_dict[station]['eta_rho'] = eta_rho
        station_dict[station]['xi_rho'] = xi_rho

    rmse_dict = calc_rmse(station_dict,constit_list)

    if print_flag == True:
        print_station_dict(station_dict,constit_list)
        print_rmse(rmse_dict,constit_list)

    return station_dict,rmse_dict


def read_atg(atg_data,site_id,constit_list):

    site_data = {}
    for key in ['site_id','name','lat','lon','amp','Gphase','reclen','delta_t','meas_type','ref']:
        site_data[key] =np.squeeze(atg_data[key][0,0][site_id-1])
    site_data['constit']=np.squeeze(atg_data['constit'][0,0][:])
    site_data['name'] = site_data['name'].strip()

    cm2m = 1.0/100.0
    for const in constit_list:

        atg_con_ind = list(site_data['constit']).index(const)
        site_data[const]=np.array([site_data['amp'][atg_con_ind]*cm2m, site_data['Gphase'][atg_con_ind]])

    return site_data

def station_ttide(zeta_da,grid,lat_t,lon_t,stime,constit_list):

    zeta_flat = zeta_da.stack(etaxi = ('eta_rho','xi_rho'))
    grid_flat = grid.stack(etaxi = ('eta_rho','xi_rho'))

    lat_s = grid_flat.lat_rho.values[grid_flat.mask_rho.values==True]
    lon_s = grid_flat.lon_rho.values[grid_flat.mask_rho.values==True]
    zeta_s = zeta_flat.values[:,grid_flat.mask_rho.values==True]
    etaxi_s = zeta_flat.etaxi.values[grid_flat.mask_rho.values==True]

    points = np.column_stack((lat_s,lon_s))
    tree = KDTree(points)

    target = np.column_stack((lat_t,lon_t))
    dist, ind = tree.query(target)
    
    lat_r = lat_s[ind]
    lon_r = lon_s[ind]
    dist = haversine(lon_t,lat_t,lon_r,lat_r)

    tmp={}
    tmp['roms_signal'] = zeta_s[:,ind].squeeze()
    
    eta_rho,xi_rho = np.fromstring(str(etaxi_s[ind])[2:-2], sep=', ',dtype=int)
    
    try:
        tmp['t_tide']=tt.t_tide(tmp['roms_signal'],dt=1,stime=stime,lat=lat_r,out_style=None)
        
        for const in constit_list:
            tide_con_ind = list(tmp['t_tide']['nameu']).index(str.encode(const+'  '))
            tmp[const]=tmp['t_tide']['tidecon'][tide_con_ind]
            
    except TypeError:
        for const in constit_list:
             tmp[const]=[np.nan,np.nan,np.nan,np.nan]

    return eta_rho,xi_rho,dist,tmp

def calc_rmse(station_dict,constit_list):

    d2r = np.pi/180
    
    const_rmse={}

    sum_complex_mse = 0
    
    sum_excluded = 0
    
    for constit in constit_list:

        tt_amp = []
        atg_amp = []

        tt_phi =[]
        atg_phi =[]

        tt_z = []
        atg_z = []

        for station,data in station_dict.items():

            if data['dist'] > 50000:              
                print('Excluded Station '+data['atg']['name']+', since dist to roms point > 50km')
                sum_excluded+=1
                
            elif np.isnan(data['tt'][constit][0]):    
                print('Excluded Station '+data['atg']['name']+', since roms ttide failed')
                sum_excluded+=1
                
            elif np.isnan(data['atg'][constit][0]):    
                print('Excluded Station '+data['atg']['name']+', since atg failed')
                sum_excluded+=1
                
            else:

                tt_amp.append(data['tt'][constit][0])
                atg_amp.append(data['atg'][constit][0])

                tt_phi.append(data['tt'][constit][2])
                atg_phi.append(data['atg'][constit][1])

                tt_z.append(data['tt'][constit][0] * np.exp(1j*data['tt'][constit][2]*d2r))
                atg_z.append(data['atg'][constit][0] * np.exp(1j*data['atg'][constit][1]*d2r))
                
        tt_amp = np.array(tt_amp)
        atg_amp=np.array(atg_amp)
        tt_phi = np.array(tt_phi)
        atg_phi = np.array(atg_phi)
        tt_z = np.array(tt_z)
        atg_z = np.array(atg_z)
        
        const_rmse[constit] = {}

        const_rmse[constit]['amp']= np.sqrt(np.mean((tt_amp - atg_amp) ** 2))
        
        phi_diff = np.abs(tt_phi-atg_phi)
        phi_diff[phi_diff>180]-=360
    
        const_rmse[constit]['phase'] = np.sqrt(np.mean(phi_diff ** 2))
        
        complex_mse = 0.5*np.mean(((tt_z-atg_z)*np.conjugate(tt_z-atg_z)).real)
        
        const_rmse[constit]['complex_amp']=np.sqrt(complex_mse)
        
        sum_complex_mse += complex_mse
        
        const_rmse[constit]['nb_excluded'] = sum_excluded

    const_rmse['combined_complex'] = np.sqrt(sum_complex_mse)
    
    return const_rmse
 
def print_station_dict(station_dict,constit_list):
    print("Station ID || Amp(amp_err)[m]:  atg   roms || phase(phase_err)[deg]:  atg   roms || Station Name; RecLen [days]; Nearest Neibour [km]")
    for constit in constit_list:
        print(constit)
        for station_id,data in station_dict.items():
            print(station_id,"|| %0.2f"%data['atg'][constit][0]," %0.2f(%0.2f) "%(data['tt'][constit][0],data['tt'][constit][1]),\
                  "|| %0.2f"%data['atg'][constit][1]," %0.2f(%0.2f) "%(data['tt'][constit][2],data['tt'][constit][3]),\
                  "|| ",data['atg']['name']," ",data['atg']['reclen'],' %0.2f' %data['dist'][0])

def print_rmse(rmse_dict,constit_list):

    for constit in constit_list:
        data = rmse_dict[constit]
        
        print(constit+' RMSD: amp = %.2f m    phase = %.2f deg   complex amp = %.2f m'%(data['amp'],data['phase'],data['complex_amp']))
        print('Number excluded records: ',rmse_dict[constit]['nb_excluded'])
        
    print('Combined complex RMSD: %.2f'%rmse_dict['combined_complex'])

In [110]:
# calculate rms
staions,rms = compare_atg(zeta,grid,stime=datetime.datetime(2007,1,15),constit_list=['M2','N2','K1','O1'],station_list=range(0,109))

stime =  2007-01-15 00:00:00  constits =  ['M2', 'N2', 'K1', 'O1'] stations =  range(0, 109)


VBox(children=(HTML(value=''), IntProgress(value=0, max=109)))

/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/ttide-0.3_exp-py3.6.egg/ttide/t_tide.py:619: RuntimeWarning: invalid value encountered in true_divide
  snr = (tidecon[:, 0] / tidecon[:, 1]) ** 2
/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/ttide-0.3_exp-py3.6.egg/ttide/t_predic.py:57: RuntimeWarning: invalid value encountered in true_divide
  snr = (tidecon[:, 0] / tidecon[:, 1]) ** 2
/home/ubuntu/bigStick/anaconda3/envs/tidal_melting/lib/python3.6/site-packages/ttide-0.3_exp-py3.6.egg/ttide/t_predic.py:60: RuntimeWarning: invalid value encountered in greater
  I = snr > synth


No predictions with this SNR
Excluded Station PTC_4_2_21, since dist to roms point > 50km
Excluded Station PTC_4_2_32, since dist to roms point > 50km
Excluded Station PTC_4_1_02 (Station "Kathy"), since dist to roms point > 50km
Excluded Station 65300- Macquarie Island, since dist to roms point > 50km
Excluded Station 20090- Heard Island, since dist to roms point > 50km
Excluded Station Beaver Lake, since dist to roms point > 50km
Excluded Station ROPEX C1, since dist to roms point > 50km
Excluded Station Rutford GL Tilt/Grav, since dist to roms point > 50km
Excluded Station Lokcroy, since roms ttide failed
Excluded Station Amery Ice Shelf TS05 (partially grounded), since dist to roms point > 50km
Excluded Station Evans Ice Stream GPS: EE2B, since dist to roms point > 50km
Excluded Station PTC_4_2_21, since dist to roms point > 50km
Excluded Station PTC_4_2_32, since dist to roms point > 50km
Excluded Station PTC_4_1_02 (Station "Kathy"), since dist to roms point > 50km
Excluded Stati

3 || 0.30  0.29(0.00)  || 92.40  97.78(0.22)  ||  Rothera   365.0  3778.04
4 || 0.15  0.14(0.00)  || 62.80  80.23(0.21)  ||  Signy   385.0  6879.25
5 || 0.29  0.24(0.00)  || 23.00  33.02(0.35)  ||  PTC_4_2_01   4.2  2610.16
6 || 0.32  0.29(0.00)  || 6.00  13.58(0.29)  ||  PTC_4_2_02   30.0  988.09
7 || 0.36  0.22(0.00)  || 17.00  9.04(0.58)  ||  PTC_4_2_03   180.0  2745.74
8 || 0.27  0.24(0.00)  || 352.00  3.63(0.20)  ||  PTC_4_2_05 Georg Von Neumayer   324.0  2270.97
9 || 0.27  0.24(0.00)  || 358.00  8.24(0.20)  ||  PTC_4_2_06 Kapp Norvegia   367.0  3475.25
10 || 0.29  0.25(0.00)  || 5.00  14.43(0.23)  ||  PTC_4_2_07 Vestkapp   316.0  1733.09
11 || 0.13  0.14(0.00)  || 93.00  103.32(0.19)  ||  PTC_4_2_12 Signy Line station 3   375.0  32728.71
12 || 0.15  0.14(0.00)  || 67.00  83.10(0.27)  ||  PTC_4_2_19 Signy Line station 5   408.0  4022.83
13 || 0.34  0.32(0.00)  || 98.00  105.68(0.26)  ||  PTC_4_2_20   357.0  7210.24
14 || 0.17  0.22(0.00)  || 97.00  94.09(0.18)  ||  PTC_4_2_21   32

44 || 0.24  0.20(0.00)  || 310.51  306.37(0.13)  ||  Syowa   365.0  10766.22
45 || 0.29  0.25(0.00)  || 45.00  43.73(0.28)  ||  Ardley Cove   44.0  4215.88
46 || 0.26  0.25(0.00)  || 50.00  46.31(0.20)  ||  Gurrachaga   41.0  4695.52
47 || 0.28  0.25(0.00)  || 49.00  46.43(0.28)  ||  Half Moon Is.   38.0  11687.06
48 || 0.29  0.24(0.00)  || 55.00  48.87(0.20)  ||  Pendulum Cove   19.0  9560.96
49 || 0.29  0.25(0.00)  || 48.00  48.68(0.20)  ||  Whalers Cove   12.0  13573.73
50 || 0.40  0.35(0.00)  || 35.00  39.68(0.29)  ||  Bahia Esperanza   547.0  3353.28
51 || 0.33  0.27(0.00)  || 52.00  53.79(0.23)  ||  Mikkelsen   25.0  9499.35
52 || 0.32  0.27(0.00)  || 59.00  54.24(0.22)  ||  Primavera   44.0  9490.28
53 || 0.30  0.27(0.00)  || 60.00  55.90(0.25)  ||  Gerlache C   29.0  4104.48
54 || 0.28  0.25(0.00)  || 65.90  63.39(0.28)  ||  Palmer   30.0  9076.80
55 || 0.29  nan(nan)  || 68.00  nan(nan)  ||  Lokcroy   19.0  9958.00
56 || 0.31  0.27(0.00)  || 63.00  55.84(0.27)  ||  Brown   699